# Connect Eye-AI and Load Libraries

In [230]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# # Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-exec"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [231]:
# Prerequisites
import json
import os
import shutil
from PIL import Image

# EyeAI, Deriva, VGG19
from deriva_ml import DatasetSpec, DatasetBag, Workflow, ExecutionConfiguration, VersionPart
from deriva_ml import MLVocab as vc
from eye_ai.eye_ai import EyeAI

# ML Analytics
import pandas as pd
import numpy as np
import sklearn.metrics as sk

# Other Utilities
from pathlib import Path, PurePath
import logging
from datetime import datetime

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [232]:
# Login
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
host = 'www.eye-ai.org'
#host = 'dev.eye-ai.org' #for dev testing
catalog_id = "eye-ai"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2025-07-30 13:47:36,230 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-07-30 13:47:36,231 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2025-07-30 13:47:36,232 - INFO - Setting up RefreshTokenAuthorizer with auth_client=[instance:140264790610640]
2025-07-30 13:47:36,233 - INFO - Setting up a RenewingAuthorizer. It will use an auth type of Bearer and can handle 401s.
2025-07-30 13:47:36,234 - INFO - RenewingAuthorizer will start by using access_token with hash "acdc0eae06d2d8d4d6aa9a666415581034840afdd37fba4f335a5cbb3f8ac109"
2025-07-30 13:47:36,234 - INFO - Executing token refresh without client credentials
2025-07-30 13:47:36,235 - INFO - Fetching new token from Globus Auth
2025-07-30 13:47:36,803 - INFO - request done (success)
2025-07-30 13:47:36,804 - INFO - RenewingAuthorizer.

You are already logged in.


# Configuration

In [233]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2025-07-30 13:47:41,718 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-07-30 13:47:41,719 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


In [234]:


#ml_instance.increment_dataset_version(dataset_rid='2-N93J', component= VersionPart.patch, description='Update to latest deriva-ml schema')

In [235]:
datasets = [ '4-411G' ]#"2-39FY" ]

data_to_download = []
for dataset in datasets:
    ds_dict = {
        'rid': dataset,
        'materialize':True,
        'version':EA.dataset_version(dataset_rid=dataset),
    }
    data_to_download.append(ds_dict)

dataset_workflow = EA.add_workflow( 
    Workflow(
        name="Make Dataset by KB",
        url="https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/Sandbox_KB/Make_Dataset.ipynb",
        workflow_type="Test Workflow",
        )
    )

config = ExecutionConfiguration(
    datasets=data_to_download,
    workflow=dataset_workflow,
    description="Test"
    )

execution = EA.create_execution(config)

2025-07-30 13:47:45,742 - INFO - Execution Aborted
Exception ignored in: <function DerivaML.__del__ at 0x7f92150b92d0>
Traceback (most recent call last):
  File "/home/kb_766/.conda/envs/my-tensorflow-conda/lib/python3.10/site-packages/deriva_ml/deriva_ml_base.py", line 158, in __del__
    self._execution.update_status(Status.aborted, "Execution Aborted")
  File "/home/kb_766/.conda/envs/my-tensorflow-conda/lib/python3.10/site-packages/pydantic/_internal/_validate_call.py", line 38, in wrapper_function
    return wrapper(*args, **kwargs)
  File "/home/kb_766/.conda/envs/my-tensorflow-conda/lib/python3.10/site-packages/pydantic/_internal/_validate_call.py", line 111, in __call__
    res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))
  File "/home/kb_766/.conda/envs/my-tensorflow-conda/lib/python3.10/site-packages/deriva_ml/execution.py", line 409, in update_status
    self._ml_object.pathBuilder.schemas[self._ml_object.ml_schema].Execution.update(


In [20]:
print(execution)

caching_dir: /data
_working_dir: /data/kb_766/EyeAI_working
execution_rid: 4-Z6K4
workflow_rid: 4-Z5Y4
asset_paths: {}
configuration: datasets=[DatasetSpec(rid='4-S42W', materialize=False, version=DatasetVersion(major=0, minor=2, patch=1))] assets=[] workflow='4-Z5Y4' parameters={} description='Instance of making a dataset; in this case, creating an angle-2 subset of 4-S42W' argv=['/home/kb_766/.conda/envs/my-tensorflow-conda/lib/python3.10/site-packages/ipykernel_launcher.py', '-f', '/home/kb_766/.local/share/jupyter/runtime/kernel-9b90ba4d-9b2b-4f49-b3ee-c52e2a09d828.json']


# Work with Data

In [236]:
ds_bag = execution.datasets[0]

In [237]:

execution._working_dir

PosixPath('/data/kb_766/EyeAI_working')

In [224]:
imageDF = ds_bag.get_table_as_dataframe('Image')[['RID', 'Filename', 'Observation']]
gradingDF = pd.read_csv('Multimodal test set grading - VN.csv')[['RID_Subject', 'Eye']]

imageDF = pd.merge( 
    imageDF,
    ds_bag.get_table_as_dataframe('Execution_Image_Fundus_Laterality')[['Image', 'Image_Side']],
    left_on = 'RID', right_on = 'Image',
    how = 'inner'
)[['RID', 'Image_Side', 'Filename', 'Observation']]

imageDF = pd.merge(
    imageDF,
    ds_bag.get_table_as_dataframe('Observation')[['RID', 'Subject']],
    left_on = 'Observation', right_on = 'RID',
    suffixes=('_Image', '_Obs'),
    how = 'left'
)

imageDF = pd.merge(
    imageDF,
    gradingDF[['RID_Subject', 'Eye']],
    left_on = ['Subject', 'Image_Side'], right_on = ['RID_Subject', 'Eye'],
    how = 'inner'
)[['RID_Subject', 'RID_Image', 'Image_Side', 'Filename']]

In [229]:
imageDF = ds_bag.get_table_as_dataframe('Image')[['RID', 'Filename', 'Observation']]
imageDF[ imageDF['RID'] == '2-7P4A' ]

,RID,Filename,Observation


In [225]:
import random
random.seed(42)
imageDF['Random_Ind'] = random.sample( range(0,len(imageDF)), len(imageDF) )
imageDF = imageDF[['Random_Ind', 'RID_Subject', 'RID_Image', 'Image_Side', 'Filename']]

In [226]:
imageDF

,Random_Ind,RID_Subject,RID_Image,Image_Side,Filename
0,81,2-7M7W,2-BDCW,Left,/data/4-411G_37ce9366c961c832931248ef462255860...
1,14,2-7M2J,2-BDHR,Left,/data/4-411G_37ce9366c961c832931248ef462255860...
2,3,2-7MEG,2-BDZG,Right,/data/4-411G_37ce9366c961c832931248ef462255860...
3,94,2-7MQC,2-BE90,Left,/data/4-411G_37ce9366c961c832931248ef462255860...
4,35,2-7MAM,2-BEA0,Left,/data/4-411G_37ce9366c961c832931248ef462255860...
...,...,...,...,...,...
113,41,2-7MTW,2-C2AJ,Left,/data/4-411G_37ce9366c961c832931248ef462255860...
114,50,2-7N3J,2-C2B0,Right,/data/4-411G_37ce9366c961c832931248ef462255860...
115,82,2-7M6A,2-C2YP,Left,/data/4-411G_37ce9366c961c832931248ef462255860...
116,112,2-7MK6,2-C382,Left,/data/4-411G_37ce9366c961c832931248ef462255860...


In [228]:
finalDF = pd.merge(
    gradingDF,
    imageDF[['RID_Subject', 'Image_Side', 'Random_Ind']],
    left_on = ['RID_Subject', 'Eye'], right_on = ['RID_Subject', 'Image_Side'],
    how = 'left'
)[['Random_Ind', 'RID_Subject', 'Eye']]

finalDF.to_csv('PairedGrading_VN.csv', index=False)

In [198]:
! rm -r -f "./Grading_KB"
! rm -r -f "./Grading_KB.zip"

In [208]:
output_path = "./Grading_KB"
os.makedirs(output_path, exist_ok= True)

In [209]:
for index, row in imageDF.iterrows():     
    src_path = row["Filename"]
    dest_name = str(row["Random_Ind"]) + "_" + row["RID_Subject"] + "_" + row["Image_Side"] + ".jpg"
    dest_path = os.path.join(output_path, dest_name)
    shutil.copy2(src_path, dest_path)

In [210]:
!ls "./Grading_KB" -1 | wc -l

117


In [211]:
ls Grading_KB

0_2-7NVC_Right.jpg    29_2-7P1W_Right.jpg  64_2-7NVA_Left.jpg
100_2-7NV0_Right.jpg  30_2-7P0C_Right.jpg  65_2-7P1W_Left.jpg
101_2-7NJ0_Right.jpg  31_2-7NWP_Left.jpg   66_2-7N88_Right.jpg
102_2-7NVT_Right.jpg  32_2-7NCG_Left.jpg   67_2-7NDP_Left.jpg
10_2-7NKA_Left.jpg    3_2-7N6W_Right.jpg   68_2-7NDP_Right.jpg
103_2-7NDJ_Left.jpg   33_2-7NQM_Right.jpg  69_2-7NQM_Left.jpg
104_2-7NT2_Right.jpg  34_2-7M2R_Left.jpg   70_2-7N5M_Left.jpg
105_2-7N9A_Right.jpg  35_2-7NB8_Right.jpg  71_2-7P1R_Left.jpg
106_2-7P38_Right.jpg  36_2-7N66_Right.jpg  72_2-7NEY_Left.jpg
107_2-7NC8_Right.jpg  37_2-7P34_Right.jpg  7_2-7NQR_Left.jpg
108_2-7NVT_Left.jpg   38_2-7N9G_Left.jpg   73_2-7NA4_Left.jpg
109_2-7NER_Right.jpg  39_2-7NXG_Left.jpg   74_2-7MZM_Right.jpg
110_2-7NS6_Right.jpg  40_2-7KZG_Right.jpg  75_2-7NB8_Left.jpg
111_2-7NQG_Left.jpg   41_2-7NKP_Right.jpg  76_2-7NV0_Left.jpg
112_2-7P12_Right.jpg  42_2-7P1R_Right.jpg  77_2-7M2R_Right.jpg
11_2-7NYC_Right.jpg   4_2-7P0T_Left.jpg    78_2-7NT4_Left.jpg
113_2

In [212]:
!zip -r Grading_KB.zip Grading_KB/

  adding: Grading_KB/ (stored 0%)
  adding: Grading_KB/81_2-7N9P_Right.jpg (deflated 9%)
  adding: Grading_KB/14_2-7P4R_Left.jpg (deflated 12%)
  adding: Grading_KB/3_2-7N6W_Right.jpg (deflated 2%)
  adding: Grading_KB/94_2-7NZ6_Right.jpg (deflated 2%)
  adding: Grading_KB/35_2-7NB8_Right.jpg (deflated 9%)
  adding: Grading_KB/31_2-7NWP_Left.jpg (deflated 4%)
  adding: Grading_KB/28_2-7NS2_Left.jpg (deflated 3%)
  adding: Grading_KB/17_2-7NAA_Right.jpg (deflated 9%)
  adding: Grading_KB/113_2-7P04_Left.jpg (deflated 7%)
  adding: Grading_KB/13_2-7NQR_Right.jpg (deflated 16%)
  adding: Grading_KB/86_2-7N88_Left.jpg (deflated 3%)
  adding: Grading_KB/108_2-7NVT_Left.jpg (deflated 2%)
  adding: Grading_KB/69_2-7NQM_Left.jpg (deflated 3%)
  adding: Grading_KB/11_2-7NYC_Right.jpg (deflated 3%)
  adding: Grading_KB/75_2-7NB8_Left.jpg (deflated 10%)
  adding: Grading_KB/54_2-7N9P_Left.jpg (deflated 11%)
  adding: Grading_KB/4_2-7P0T_Left.jpg (deflated 4%)
  adding: Grading_KB/114_2-7NXW_Right

In [ ]:
# Buffer